# MIMIC Notes Pre-Processing

Pre-processing MIMIC notes for language model and CUIs

## Imports and Inits

In [1]:
import pandas as pd
import psycopg2
import numpy as np
import re
import random

from pathlib import Path

In [2]:
PATH = Path('data')

## Functions

## Grab sample data from MIMIC

In [3]:
cats = pd.read_csv('cats.csv')
max_limit = 1000

queries = []
for category, n_notes in zip(cats['category'], cats['number_of_notes']):
    limit = min(max_limit, n_notes) if max_limit > 0 else n_notes
    if limit == max_limit:
        q = f"""
        select category, text from correctnotes where category=\'{category}\' order by random() limit {limit};
        """
    else:
        q = f"""
        select category, text from correctnotes where category=\'{category}\';
        """
    queries.append(q)

In [4]:
# limit = 50
# queries = [
#     f"""
#     select category, text from correctnotes where category=\'{cats.iloc[7]['category']}\' order by random() limit {limit}
#     """
# ]

In [5]:
%%time
dfs = []

con = psycopg2.connect(dbname='mimic', user='sudarshan', host='/var/run/postgresql')
for q in queries:
    df = pd.read_sql_query(q, con)
    dfs.append(df)
con.close()
    
df = pd.concat(dfs)
df.reset_index(inplace=True, drop=True)
# df.set_index('row_id', inplace=True)
print(df.shape)

(12152, 2)
CPU times: user 85.8 ms, sys: 48.6 ms, total: 134 ms
Wall time: 13.5 s


1. Get list of redacted types using `re`
2. replace it with appropriate holder tokens

Below is a list of redacted items with an example and the replacement token.

Redacted items:
* [x] First Name: `[**First Name (Titles) 137**]`, `t_firstname`
* [x] Last Name: `[**Last Name (Titles) **]`, `t_lastname`
* [x] Initials: `[**Initials (NamePattern4) **]`, `t_initials`
* [x] Name: `[**Name (NI) **]`, `t_name`
* [x] Doctor First Name: `[**Doctor First Name 1266**]`, `t_doctor_firstname`
* [x] Doctor Last Name: `[**Doctor Last Name 1266**]`, `t_doctor_lastname`
* [x] Known Last Name: `[**Known lastname 658**]`, `t_lastname`
* [x] Hospital: `[**Hospital1 **]`, `t_hospital`
* [x] Hospital Unit Name: `**Hospital Unit Name 10**`, `t_hospital`
* [x] Company: `[**Company 12924**]`, `t_workplace`
* [x] University/College: `[**University/College **]`, `t_workplace`
* [x] Date of format YYYY-M-DD: `[**2112-4-18**]`, `t_fulldate`
* [x] Year: `[**Year (4 digits) **]`, `t_year`
* [x] Year YYYY format: `[**2119**]`, `t_year` - I use a regex `\b\d{4}\b` that will match **any** 4 digits which might be problematic, but for the most part 4 digits by itself seems to indicate a year.
* [x] Date of format M-DD: `[**6-12**]`, `[**12/2151**]`, `t_monthday`
* [x] Month/Day: `[**Month/Day (2) 509**]`, `t_monthday`
* [x] Month (only): `[**Month (only) 51**]`, `t_month`
* [x] Holiday: `[**Holiday 3470**]`, `t_month`
* [x] Date Range: `[**Date range (1) 7610**]`, `t_daterange`
* [x] Country: `[**Country 9958**]`, `t_country`
* [x] State: `[**State 3283**]`, `t_state`
* [x] Location: `**Location (un) 2432**`, `t_location`
* [x] Telephone/Fax: `[**Telephone/Fax (3) 8049**]`, `t_phone`
* [x] Clip Number: `[**Clip Number (Radiology) 29923**]`, `t_radclip_id`
* [x] Pager Numeric Identifier: `[**Numeric Identifier 6403**]`, `t_pager_id`
* [x] Pager Number: `[**Pager number 13866**]`, `t_pager_id`
* [x] Social Security Number: `[**Security Number 10198**]`, `t_ssn`
* [x] Serial Number: `[**Serial Number 3567**]`, `t_sn`
* [x] Medical Record Number: `[**Medical Record Number **]`, `t_mrn`
* [x] Provider Number: `[**Provider Number 12521**]`, `t_provider_no`
* [x] Age over 90: `[**Age over 90 **]`, `t_oldage`
* Just numbers: `[** 7901**]`
* Wardname
* Pharmacy MD Number* 

In [6]:
pat = re.compile(r'\[\*\*(.*?)\*\*\]', re.IGNORECASE)

In [59]:
def redacorator(func):
    def replace(match):
        ori = match.group()
        text = match.group().lower()
        if set(ori) == set(' *]['):
            ori = ''
        return func(text, ori)
    return replace

@redacorator
def replace_name(text, ori):
    r = ori
    if 'name' in text:
        r = 't_name'
        if 'last' in text:
            if 'doctor' in text:
                r = 't_doctor_lastname'
            else:
                r = 't_lastname'
        elif 'first' in text:
            if 'doctor' in text:
                r = 't_doctor_firstname'
            else:
                r = 't_firstname'
        elif 'initials' in text:
            r = 't_initials'
    return r

@redacorator
def replace_place(text, ori):
    r = ori
    if 'hospital' in text:
        r = 't_hospital'
    elif ('company' in text) or ('university/college' in text):
        r = 't_workplace'
    elif 'location' in text:
        r = 't_location'
    elif 'country' in text:
        r = 't_country'
    elif 'state' in text:
        r = 't_state'
    elif ('address' in text) or ('po box' in text):
        r = 't_address'
    return r

@redacorator
def replace_dates(text, ori):
    r = ori
    if 'year' in text or re.search(r'\b\d{4}\b', text):
        r = 't_year'
    elif re.search(r'\d{4}-\d{0,2}-\d{0,2}', text):
        r = 't_fulldate'        
    elif (re.search(r'\d{0,2}-\d{0,2}', text)) or (re.search(r'\d{0,2}\/\d{0,2}', text)) or ('month/day' in text):
        r = 't_monthday'
    elif 'month' in text:
        r = 't_month'
    elif 'holiday' in text:
        r = 't_holiday'
    elif 'date range' in text:
        r = 't_daterange'
    return r

@redacorator
def replace_identifiers(text, ori):
    r = ori
    if ('numeric identifier' in text) or ('pager number' in text):
        r = 't_pager_id'
    elif '(radiology)' in text:
        r = 't_radclip_id'
    elif 'social security number' in text:
        r = 't_ssn'
    elif 'medical record number' in text:
        r = 't_mrn'
    elif 'age over 90' in text:
        r = 't_oldage'
    elif 'serial number' in text:
        r = 't_sn'
    elif 'unit number' in text:
        r = 't_unit_no'
    elif 'md number' in text:
        r = 't_md_no'
    elif 'telephone/fax' in text:
        r = 't_phone'
    elif 'provider number' in text:
        r = 't_provider_no'
    return r

In [168]:
def replace_redacted(text):
    pat = re.compile(r'\[\*\*(.*?)\*\*\]', re.IGNORECASE)
    
    # replace name types
    text = pat.sub(replace_name, text)
    
    # replace place types
    text = pat.sub(replace_place, text)
    
    # replace date types
    text = pat.sub(replace_dates, text)

    # replace person identifier types
    text = pat.sub(replace_identifiers, text)

    text = re.sub(r'\[\*\*(\d{2})\*\*\] \b[A|P].?M.?\b', 't_hour', text, re.IGNORECASE) 
    
    return text

def misc_scrub(text):
    # replace different types of "year old" with year_old
    # matches: y.o., y/o, years old. year old, yearold
    text = re.sub(r'\byears? ?old\b|\by(?:o|r)*[ ./-]*o(?:ld)?\b', 't_year_old',
               text, flags=re.IGNORECASE)
    
    # replaces yr, yr's, yrs with years
    text = re.sub(r'\byr[\'s]*\b', 'years', text, re.IGNORECASE)
    
    # replace Pt and pt with patient, and IN/OUT/OT PT with patient
    # Note: PT also refers to physical therapy and physical therapist
    text = re.sub(r'\b[P|p]t.?|\b(IN|OU?T) PT\b', 'patient', text)
    
    return text

In [169]:
def scrub_text(text):
    # replace redacted info with tokens
    text = replace_redacted(text)
    
    # misc scrubbing
    text = misc_scrub(text)
    
    return text

In [170]:
pat2 = re.compile(r'\[\*\*(\d{2})\*\*\] \b[A|P].?M.?\b', re.IGNORECASE)

In [171]:
test = df.iloc[3482]['text']
# for m in pat.finditer(test):
#     print(m)

In [172]:
print(test)

CVICU
   HPI:
   HD5
   [**12-17**] POD [**3-17**]
   63M s/p Repair of Type A dissection with replacement of ascending
   aorta(26mm Gelweave)/AVR(25mm porcine) [**12-13**], RLE fasciotomies [**12-15**]
   EF: 55% Wt.: 72.7 kg Cr.:1.2
   PMHx: HTN, GERD,
   [**Last Name (un) **]: Atenolol
   Current medications:
   Acetaminophen, Aspirin EC, Calcium Gluconate, Docusate Sodium, Insulin,
   Lisinopril, Metoprolol Tartrate, Metoprolol Tartrate, Metoclopramide,
   Morphine Sulfate, NiCARdipine, Nitroglycerin, Oxycodone-Acetaminophen,
   Potassium Chloride, Ranitidine
   24 Hour Events:
 CCO PAC - STOP [**2134-12-16**] 10:00 AM
 CORDIS/INTRODUCER - START [**2134-12-16**] 10:21 AM
   placed in OR [**12-13**]
 TEMPORARY PACEMAKER WIRES DISCONTINUED - At [**2134-12-16**] 11:10 AM
 CHEST TUBE REMOVED - At [**2134-12-16**] 11:15 AM
 INVASIVE VENTILATION - STOP [**2134-12-16**] 05:27 PM
 EXTUBATION - At [**2134-12-16**] 05:29 PM
extubated
post-op htn
 remains on NTG gtt for hypertension
   Post 

In [174]:
out = pat2.sub('t_hour', test)

In [173]:
for i, row in df.iterrows():
    if len(pat2.findall(row['text'])) != 0:
        print(i, pat2.findall(row['text']))

968 ['57']
970 ['30']
971 ['57']
1007 ['10']
1008 ['08']
1010 ['65']
1016 ['01']
1018 ['71']
1019 ['16']
1022 ['16']
1023 ['01']
1024 ['52']
1025 ['85']
1026 ['52']
1046 ['23']
1047 ['88']
1048 ['88']
1049 ['70']
3059 ['55']
3063 ['64']
3065 ['64']
3120 ['27']
3141 ['08']
3170 ['70']
3186 ['47']
3189 ['49']
3198 ['33']
3225 ['08']
3242 ['74']
3243 ['26']
3247 ['84']
3299 ['19']
3301 ['81']
3323 ['60']
3330 ['76']
3353 ['42']
3360 ['21']
3374 ['41']
3422 ['20']
3482 ['37']
3516 ['06']
3519 ['02']
3523 ['30']
3534 ['50']
3562 ['39']
3571 ['77']
3589 ['04']
3609 ['93']
3618 ['11']
3676 ['04']
3688 ['12']
3781 ['14']
3786 ['17']
3790 ['04']
3793 ['55']
3814 ['72']
3827 ['78']
3828 ['58']
3837 ['50']
3868 ['91']
3906 ['55']
3909 ['68']
3916 ['43']
3924 ['48']
3940 ['82']
3991 ['97']
3993 ['78']
4050 ['50']
7118 ['82']
7133 ['97']
7139 ['21']
7153 ['27']
7157 ['46']
7158 ['51']
7164 ['72']
7170 ['56']
7174 ['46']
7177 ['11']
7178 ['90']
7181 ['19']
7183 ['48']
7185 ['33']
7186 ['34']
7187 ['

In [106]:
for m in pat2.finditer(test):
    print(m)

In [74]:
out = scrub_text(test)
# for m in pat.finditer(out):
#     print(m)

In [ ]:
print(out)

In [161]:
df['scrubbed'] = df['text'].apply(scrub_text)

In [162]:
for i, row in df.iterrows():
    if len(pat.findall(row['scrubbed'])) != 0:
        print(i, pat.findall(row['scrubbed']))

590 [' 122']
691 ['60']
2078 [' 324']
2908 [' 16']
3059 ['55']
3063 ['64']
3065 ['64']
3075 [' 522']
3120 ['27']
3189 ['49']
3198 ['15', '18', '33']
3243 ['26']
3299 ['19']
3330 ['76']
3360 ['21']
3422 ['20']
3482 ['37']
3501 ['89']
3506 ['07', '07', '91']
3516 ['87', '96', '06']
3519 ['02']
3534 ['50']
3562 ['39']
3571 ['77']
3589 ['04']
3609 ['93']
3676 ['04']
3688 ['12']
3766 ['21']
3781 ['14']
3786 ['17']
3790 ['04']
3793 ['55']
3814 ['72']
3828 ['58']
3837 ['50']
3868 ['91']
3906 ['55']
3909 ['68']
3924 ['48']
3940 ['82']
3991 ['97']
3993 ['78']
4050 ['50']
4104 ['08']
4140 ['62']
4214 ['47', '77', '68', '77', '47']
4248 ['40', '23']
4309 ['23']
4616 ['43']
4625 ['36']
4712 ['77']
4826 [' 494', ' 494']
4878 ['47']
4880 ['02']
4901 ['22']
5012 ['60']
5025 ['74']
5051 [' 467']
5065 [' 217']
5068 [' 83']
5074 [' 18']
5079 [' 685']
5089 [' 529', ' 529']
5104 [' 29']
5106 [' 227']
5109 ['50']
5118 [' 435']
5142 [' 37']
5144 [' 543']
5146 [' 83', ' 21', ' 14']
5188 [' 32']
5189 [' 79']
